In [39]:
import os
import cv2
import torch
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip

from PIL import Image, ImageDraw, ImageFont

In [40]:
# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5x')

Using cache found in C:\Users\Rabeea/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-10-25 Python-3.9.16 torch-1.11.0+cpu CPU

Fusing layers... 
YOLOv5x summary: 444 layers, 86705005 parameters, 0 gradients
Adding AutoShape... 


In [41]:
# Video Folder Path
video_path = r'D:\MLOps\first 10'

# Reading Excel File
data = pd.read_excel(r'D:\MLOps\Demo_trail\Demo\kikkoman_dashboard.xlsx', sheet_name = 0)

In [42]:
def filename_to_seconds(file):

    time_str = os.path.splitext(file)[0]

    # Convert the string to a datetime object
    time_obj = datetime.strptime(time_str, "%H%M%S")

    # Format the datetime object to the desired format
    formatted_time = time_obj.strftime("%H:%M:%S")
    
    return formatted_time

def format_time(time_obj):
    """Convert a datetime.time object to a formatted string in the HH:MM:SS format."""
    return time_obj.strftime("%H:%M:%S")

def get_sec(time_str):
    """Convert a time string to a datetime.time object then get seconds."""
    return datetime.strptime(time_str, "%H:%M:%S").second

# Function to draw data for each tank
def draw_tank_data(tank_name, productive_time, idle_time, y_position):
    # Draw white background for the row
    draw.rectangle([(x_position, y_position - 20), (x_position + 300, y_position + 10)], fill="white")
    # Display the data in black
    draw.text((x_position, y_position), tank_name, font=font, fill="black")
    draw.text((x_position + 100, y_position), productive_time, font=font, fill="black")
    draw.text((x_position + 200, y_position), idle_time, font=font, fill="black")
    return y_position + 30

In [43]:
# Setting File format
data['Start_File'] = [filename_to_seconds(str(obj)) for obj in data['Start_File']]

In [44]:
tank_roi_dict = {
    "Tank 5": [(384, 228), (472, 239), (407, 377), (300, 437), (230, 386), (326, 345)],
    "Tank 6": [(472, 239), (611, 256), (561, 415), (400, 495), (300, 437), (407, 377)],
    "Tank 7": [(611, 256), (858, 286), (796, 488), (643, 613), (400, 495), (561, 415)],
    "Tank 8": [(858, 286), (1203, 328), (1126, 679), (821, 679), (643, 613), (796, 488)],
    "PLC": [(30, 275), (183, 275), (220, 509), (51, 509)]
}

In [45]:
# # setting total frames for all tanks in sec
# total_tank5_idle_frame = 0
# total_tank5_productive_frame = 0
# total_tank6_idle_frame = 0
# total_tank6_productive_frame = 0
# total_tank7_idle_frame = 0
# total_tank7_productive_frame = 0
# total_tank8_idle_frame = 0
# total_tank8_productive_frame = 0
# total_plc_idle_frame = 0
# total_plc_productive_frame = 0

# for file in os.listdir(video_path):

#     # Reading video file
#     video_file_path = os.path.join(video_path, file)
#     cap = cv2.VideoCapture(video_file_path)

#     file_name = filename_to_seconds(file)

#     # Get the frames per second (fps) of the video
#     fps = int(cap.get(cv2.CAP_PROP_FPS))

#     # setting current frames for all tanks
#     current_frame = 0
#     tank5_idle_frame = 0
#     tank5_productive_frame = 0
#     tank6_idle_frame = 0
#     tank6_productive_frame = 0
#     tank7_idle_frame = 0
#     tank7_productive_frame = 0
#     tank8_idle_frame = 0
#     tank8_productive_frame = 0
#     plc_idle_frame = 0
#     plc_productive_frame = 0

#     while True:
#         ret, frame = cap.read()
#         if not ret:
#             break
        
#         current_frame += 1

#         frame = cv2.resize(frame, (1420, 680))

#         results = model(frame)

#         # Draw rectangles around detected persons
#         for *xyxy, conf, cls in results.pred[0]:
#             if int(cls) == 0:  # Assuming class ID for person is 0
#                     cv2.rectangle(frame, (int(xyxy[0]), int(xyxy[1])), (int(xyxy[2]), int(xyxy[3])), (0, 255, 0), 2)

#         for tank, roi_points in tank_roi_dict.items():
#             roi_pts = np.array(roi_points, np.int32)
#             cv2.polylines(frame, [roi_pts], isClosed=True, color=(0, 255, 0), thickness=2)

#             if tank == 'Tank 5':
    
#                 temp = data[(data['Start_File'] == file_name) & (data['Station'] == tank)]

#                 if len(temp) > 0:

#                     start_frame = temp['Start_Time'].values[0] * fps
#                     end_frame = temp['End_Time'].values[0] * fps

#                     if start_frame <= current_frame <= end_frame:
#                         tank5_productive_frame += 1
#                     else:
#                         tank5_idle_frame += 1
#                 else:
#                     tank5_idle_frame += 1

#                 # Calculate the current idle time for this frame
#                 current_tank5_idle_time_seconds = total_tank5_idle_frame + (tank5_idle_frame // fps)
#                 tank5_idle_time_display = f"{current_tank5_idle_time_seconds // 60:02}:{current_tank5_idle_time_seconds % 60:02}"

#                 # Calculate the current productive time for this frame
#                 current_tank5_productive_time_seconds = total_tank5_productive_frame + (tank5_productive_frame // fps)
#                 tank5_productive_time_display = f"{current_tank5_productive_time_seconds // 60:02}:{current_tank5_productive_time_seconds % 60:02}"

#             elif tank == 'Tank 6':

#                 temp = data[(data['Start_File'] == file_name) & (data['Station'] == tank)]

#                 if len(temp) > 0:

#                     start_frame = temp['Start_Time'].values[0] * fps
#                     end_frame = temp['End_Time'].values[0] * fps

#                     if start_frame <= current_frame <= end_frame:
#                         tank6_productive_frame += 1
#                     else:
#                         tank6_idle_frame += 1
#                 else:
#                     tank6_idle_frame += 1

#                 # Calculate the current idle time for this frame
#                 current_tank6_idle_time_seconds = total_tank6_idle_frame + (tank6_idle_frame // fps)
#                 tank6_idle_time_display = f"{current_tank6_idle_time_seconds // 60:02}:{current_tank6_idle_time_seconds % 60:02}"

#                 # Calculate the current productive time for this frame
#                 current_tank6_productive_time_seconds = total_tank6_productive_frame + (tank6_productive_frame // fps)
#                 tank6_productive_time_display = f"{current_tank6_productive_time_seconds // 60:02}:{current_tank6_productive_time_seconds % 60:02}"

#             elif tank == 'Tank 7':

#                 temp = data[(data['Start_File'] == file_name) & (data['Station'] == tank)]

#                 if len(temp) > 0:

#                     start_frame = temp['Start_Time'].values[0] * fps
#                     end_frame = temp['End_Time'].values[0] * fps

#                     if start_frame <= current_frame <= end_frame:
#                         tank7_productive_frame += 1
#                     else:
#                         tank7_idle_frame += 1

#                 else:
#                     tank7_idle_frame += 1

#                 # Calculate the current idle time for this frame
#                 current_tank7_idle_time_seconds = total_tank7_idle_frame + (tank7_idle_frame // fps)
#                 tank7_idle_time_display = f"{current_tank7_idle_time_seconds // 60:02}:{current_tank7_idle_time_seconds % 60:02}"

#                 # Calculate the current productive time for this frame
#                 current_tank7_productive_time_seconds = total_tank7_productive_frame + (tank7_productive_frame // fps)
#                 tank7_productive_time_display = f"{current_tank7_productive_time_seconds // 60:02}:{current_tank7_productive_time_seconds % 60:02}"

#             elif tank == 'Tank 8':

#                 temp = data[(data['Start_File'] == file_name) & (data['Station'] == tank)]

#                 if len(temp) > 0:

#                     start_frame = temp['Start_Time'].values[0] * fps
#                     end_frame = temp['End_Time'].values[0] * fps

#                     if start_frame <= current_frame <= end_frame:
#                         tank8_productive_frame += 1
#                     else:
#                         tank8_idle_frame += 1
                
#                 else:
#                     tank8_idle_frame += 1

#                 # Calculate the current idle time for this frame
#                 current_tank8_idle_time_seconds = total_tank8_idle_frame + (tank8_idle_frame // fps)
#                 tank8_idle_time_display = f"{current_tank8_idle_time_seconds // 60:02}:{current_tank8_idle_time_seconds % 60:02}"

#                 # Calculate the current productive time for this frame
#                 current_tank8_productive_time_seconds = total_tank8_productive_frame + (tank8_productive_frame // fps)
#                 tank8_productive_time_display = f"{current_tank8_productive_time_seconds // 60:02}:{current_tank8_productive_time_seconds % 60:02}"

#             elif tank == 'PLC':

#                 temp = data[(data['Start_File'] == file_name) & (data['Station'] == tank)]

#                 if len(temp) > 0:

#                     start_frame = temp['Start_Time'].values[0] * fps
#                     end_frame = temp['End_Time'].values[0] * fps

#                     if start_frame <= current_frame <= end_frame:
#                         plc_productive_frame += 1
#                     else:
#                         plc_idle_frame += 1

#                 else:
#                     plc_idle_frame += 1

#                 # Calculate the current idle time for this frame
#                 current_plc_idle_time_seconds = total_plc_idle_frame + (plc_idle_frame // fps)
#                 plc_idle_time_display = f"{current_plc_idle_time_seconds // 60:02}:{current_plc_idle_time_seconds % 60:02}"

#                 # Calculate the current productive time for this frame
#                 current_plc_productive_time_seconds = total_plc_productive_frame + (plc_productive_frame // fps)
#                 plc_productive_time_display = f"{current_plc_productive_time_seconds // 60:02}:{current_plc_productive_time_seconds % 60:02}"

#         # Convert the OpenCV frame to a PIL Image
#         pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
#         draw = ImageDraw.Draw(pil_image)
#         font = ImageFont.load_default()

#         # Define starting x_position and y_position
#         x_position = 0
#         y_position = 25

#         # Draw a white rectangle for the header background
#         header_height = 25
#         draw.rectangle([(x_position, y_position - header_height), (x_position + 300, y_position)], fill="white")

#         # Display the column names in blue
#         draw.text((x_position, y_position - 20), "Tank", font=font, fill=(0, 0, 255))
#         draw.text((x_position + 100, y_position - 20), "Productive Time", font=font, fill=(0, 0, 255))
#         draw.text((x_position + 200, y_position - 20), "Idle Time", font=font, fill=(0, 0, 255))

#         # Update y_position for the tank data
#         y_position += 30

#         # Display the data for each tank and PLC
#         y_position = draw_tank_data("Tank 5", tank5_productive_time_display, tank5_idle_time_display, y_position)
#         y_position = draw_tank_data("Tank 6", tank6_productive_time_display, tank6_idle_time_display, y_position)
#         y_position = draw_tank_data("Tank 7", tank7_productive_time_display, tank7_idle_time_display, y_position)
#         y_position = draw_tank_data("Tank 8", tank8_productive_time_display, tank8_idle_time_display, y_position)
#         y_position = draw_tank_data("PLC", plc_productive_time_display, plc_idle_time_display, y_position)
    

#         frame = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)
#         cv2.imshow('Detections with Productive Time', frame)

#         if cv2.waitKey(25) & 0xFF == ord('q'):
#             break

#     # After processing each video, add the productive time from this video to the total productive time
#     total_tank5_productive_frame += tank5_productive_frame // fps
#     total_tank5_idle_frame += tank5_idle_frame // fps
#     total_tank6_productive_frame += tank6_productive_frame // fps
#     total_tank6_idle_frame += tank6_idle_frame // fps
#     total_tank7_productive_frame += tank7_productive_frame // fps
#     total_tank7_idle_frame += tank7_idle_frame // fps
#     total_tank8_productive_frame += tank8_productive_frame // fps
#     total_tank8_idle_frame += tank8_idle_frame // fps
#     total_plc_productive_frame += plc_productive_frame // fps
#     total_plc_idle_frame += plc_idle_frame // fps

# cap.release()
# cv2.destroyAllWindows() 

In [48]:
def draw_tank_text_on_roi(tank_name, productive_time, idle_time, roi_points, frame):
    # Calculate a suitable top-left corner for the text based on the ROI's coordinates.
    x_position = min(pt[0] for pt in roi_points)
    y_position = min(pt[1] for pt in roi_points) - 30  # Offset it a bit above the ROI
    
    # Draw the tank name, productive time, and idle time on the frame.
    cv2.putText(frame, tank_name, (x_position, y_position), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    cv2.putText(frame, productive_time, (x_position, y_position + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    cv2.putText(frame, idle_time, (x_position, y_position + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

In [49]:
# setting total frames for all tanks in sec
total_tank5_idle_frame = 0
total_tank5_productive_frame = 0
total_tank6_idle_frame = 0
total_tank6_productive_frame = 0
total_tank7_idle_frame = 0
total_tank7_productive_frame = 0
total_tank8_idle_frame = 0
total_tank8_productive_frame = 0
total_plc_idle_frame = 0
total_plc_productive_frame = 0

for file in os.listdir(video_path):

    # Reading video file
    video_file_path = os.path.join(video_path, file)
    cap = cv2.VideoCapture(video_file_path)

    file_name = filename_to_seconds(file)

    # Get the frames per second (fps) of the video
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # setting current frames for all tanks
    current_frame = 0
    tank5_idle_frame = 0
    tank5_productive_frame = 0
    tank6_idle_frame = 0
    tank6_productive_frame = 0
    tank7_idle_frame = 0
    tank7_productive_frame = 0
    tank8_idle_frame = 0
    tank8_productive_frame = 0
    plc_idle_frame = 0
    plc_productive_frame = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        current_frame += 1

        frame = cv2.resize(frame, (1420, 680))

        results = model(frame)

        # Draw rectangles around detected persons
        for *xyxy, conf, cls in results.pred[0]:
            if int(cls) == 0:  # Assuming class ID for person is 0
                    cv2.rectangle(frame, (int(xyxy[0]), int(xyxy[1])), (int(xyxy[2]), int(xyxy[3])), (0, 255, 0), 2)

        for tank, roi_points in tank_roi_dict.items():
            roi_pts = np.array(roi_points, np.int32)
            cv2.polylines(frame, [roi_pts], isClosed=True, color=(0, 255, 0), thickness=2)

            if tank == 'Tank 5':
                draw_tank_text_on_roi(tank, tank5_productive_time_display, tank5_idle_time_display, roi_points, frame)
                temp = data[(data['Start_File'] == file_name) & (data['Station'] == tank)]

                if len(temp) > 0:

                    start_frame = temp['Start_Time'].values[0] * fps
                    end_frame = temp['End_Time'].values[0] * fps

                    if start_frame <= current_frame <= end_frame:
                        tank5_productive_frame += 1
                    else:
                        tank5_idle_frame += 1
                else:
                    tank5_idle_frame += 1

                # Calculate the current idle time for this frame
                current_tank5_idle_time_seconds = total_tank5_idle_frame + (tank5_idle_frame // fps)
                tank5_idle_time_display = f"{current_tank5_idle_time_seconds // 60:02}:{current_tank5_idle_time_seconds % 60:02}"

                # Calculate the current productive time for this frame
                current_tank5_productive_time_seconds = total_tank5_productive_frame + (tank5_productive_frame // fps)
                tank5_productive_time_display = f"{current_tank5_productive_time_seconds // 60:02}:{current_tank5_productive_time_seconds % 60:02}"

            elif tank == 'Tank 6':
                draw_tank_text_on_roi(tank, tank6_productive_time_display, tank6_idle_time_display, roi_points, frame)
                temp = data[(data['Start_File'] == file_name) & (data['Station'] == tank)]

                if len(temp) > 0:

                    start_frame = temp['Start_Time'].values[0] * fps
                    end_frame = temp['End_Time'].values[0] * fps

                    if start_frame <= current_frame <= end_frame:
                        tank6_productive_frame += 1
                    else:
                        tank6_idle_frame += 1
                else:
                    tank6_idle_frame += 1

                # Calculate the current idle time for this frame
                current_tank6_idle_time_seconds = total_tank6_idle_frame + (tank6_idle_frame // fps)
                tank6_idle_time_display = f"{current_tank6_idle_time_seconds // 60:02}:{current_tank6_idle_time_seconds % 60:02}"

                # Calculate the current productive time for this frame
                current_tank6_productive_time_seconds = total_tank6_productive_frame + (tank6_productive_frame // fps)
                tank6_productive_time_display = f"{current_tank6_productive_time_seconds // 60:02}:{current_tank6_productive_time_seconds % 60:02}"

            elif tank == 'Tank 7':
                draw_tank_text_on_roi(tank, tank7_productive_time_display, tank7_idle_time_display, roi_points, frame)
                temp = data[(data['Start_File'] == file_name) & (data['Station'] == tank)]

                if len(temp) > 0:

                    start_frame = temp['Start_Time'].values[0] * fps
                    end_frame = temp['End_Time'].values[0] * fps

                    if start_frame <= current_frame <= end_frame:
                        tank7_productive_frame += 1
                    else:
                        tank7_idle_frame += 1

                else:
                    tank7_idle_frame += 1

                # Calculate the current idle time for this frame
                current_tank7_idle_time_seconds = total_tank7_idle_frame + (tank7_idle_frame // fps)
                tank7_idle_time_display = f"{current_tank7_idle_time_seconds // 60:02}:{current_tank7_idle_time_seconds % 60:02}"

                # Calculate the current productive time for this frame
                current_tank7_productive_time_seconds = total_tank7_productive_frame + (tank7_productive_frame // fps)
                tank7_productive_time_display = f"{current_tank7_productive_time_seconds // 60:02}:{current_tank7_productive_time_seconds % 60:02}"

            elif tank == 'Tank 8':
                draw_tank_text_on_roi(tank, tank8_productive_time_display, tank8_idle_time_display, roi_points, frame)
                temp = data[(data['Start_File'] == file_name) & (data['Station'] == tank)]

                if len(temp) > 0:

                    start_frame = temp['Start_Time'].values[0] * fps
                    end_frame = temp['End_Time'].values[0] * fps

                    if start_frame <= current_frame <= end_frame:
                        tank8_productive_frame += 1
                    else:
                        tank8_idle_frame += 1
                
                else:
                    tank8_idle_frame += 1

                # Calculate the current idle time for this frame
                current_tank8_idle_time_seconds = total_tank8_idle_frame + (tank8_idle_frame // fps)
                tank8_idle_time_display = f"{current_tank8_idle_time_seconds // 60:02}:{current_tank8_idle_time_seconds % 60:02}"

                # Calculate the current productive time for this frame
                current_tank8_productive_time_seconds = total_tank8_productive_frame + (tank8_productive_frame // fps)
                tank8_productive_time_display = f"{current_tank8_productive_time_seconds // 60:02}:{current_tank8_productive_time_seconds % 60:02}"

            elif tank == 'PLC':
                draw_tank_text_on_roi(tank, plc_productive_time_display, plc_idle_time_display, roi_points, frame)
                temp = data[(data['Start_File'] == file_name) & (data['Station'] == tank)]

                if len(temp) > 0:

                    start_frame = temp['Start_Time'].values[0] * fps
                    end_frame = temp['End_Time'].values[0] * fps

                    if start_frame <= current_frame <= end_frame:
                        plc_productive_frame += 1
                    else:
                        plc_idle_frame += 1

                else:
                    plc_idle_frame += 1

                # Calculate the current idle time for this frame
                current_plc_idle_time_seconds = total_plc_idle_frame + (plc_idle_frame // fps)
                plc_idle_time_display = f"{current_plc_idle_time_seconds // 60:02}:{current_plc_idle_time_seconds % 60:02}"

                # Calculate the current productive time for this frame
                current_plc_productive_time_seconds = total_plc_productive_frame + (plc_productive_frame // fps)
                plc_productive_time_display = f"{current_plc_productive_time_seconds // 60:02}:{current_plc_productive_time_seconds % 60:02}"

        # Convert the OpenCV frame to a PIL Image
        pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        draw = ImageDraw.Draw(pil_image)
        font = ImageFont.load_default()

        # Define starting x_position and y_position
        x_position = 0
        y_position = 25

        # Draw a white rectangle for the header background
        header_height = 25
        draw.rectangle([(x_position, y_position - header_height), (x_position + 300, y_position)], fill="white")

        # Display the column names in blue
        draw.text((x_position, y_position - 20), "Tank", font=font, fill=(0, 0, 255))
        draw.text((x_position + 100, y_position - 20), "Productive Time", font=font, fill=(0, 0, 255))
        draw.text((x_position + 200, y_position - 20), "Idle Time", font=font, fill=(0, 0, 255))

        # Update y_position for the tank data
        y_position += 30

        # Display the data for each tank and PLC
        y_position = draw_tank_data("Tank 5", tank5_productive_time_display, tank5_idle_time_display, y_position)
        y_position = draw_tank_data("Tank 6", tank6_productive_time_display, tank6_idle_time_display, y_position)
        y_position = draw_tank_data("Tank 7", tank7_productive_time_display, tank7_idle_time_display, y_position)
        y_position = draw_tank_data("Tank 8", tank8_productive_time_display, tank8_idle_time_display, y_position)
        y_position = draw_tank_data("PLC", plc_productive_time_display, plc_idle_time_display, y_position)
    

        frame = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)
        cv2.imshow('Detections with Productive Time', frame)

        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    # After processing each video, add the productive time from this video to the total productive time
    total_tank5_productive_frame += tank5_productive_frame // fps
    total_tank5_idle_frame += tank5_idle_frame // fps
    total_tank6_productive_frame += tank6_productive_frame // fps
    total_tank6_idle_frame += tank6_idle_frame // fps
    total_tank7_productive_frame += tank7_productive_frame // fps
    total_tank7_idle_frame += tank7_idle_frame // fps
    total_tank8_productive_frame += tank8_productive_frame // fps
    total_tank8_idle_frame += tank8_idle_frame // fps
    total_plc_productive_frame += plc_productive_frame // fps
    total_plc_idle_frame += plc_idle_frame // fps

cap.release()
cv2.destroyAllWindows() 

In [46]:
cv2.destroyAllWindows() 